In [35]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, ElementNotInteractableException, TimeoutException, StaleElementReferenceException, ElementClickInterceptedException
from webdriver_manager.chrome import ChromeDriverManager

import time
import re
import pandas as pd

from tqdm.notebook import tqdm
from collections import defaultdict

from bs4 import BeautifulSoup
import requests

## Что нужно собрать:
* Гиперссылка на проект
* Название проекта
* Организация
* Конкурс
* Запрашиваемая сумма
* Cофинансирование
* Общая сумма расходов на реализацию проекта
* Цель
* Краткое описание
* Статус проекта
* Номер заявки (индекс таблицы)
* Дата подачи
* Сроки реализации
* ИНН
* ОГРН

In [2]:
start_url = 'https://президентскиегранты.рф/public/application/cards?SearchString=&Statuses%5B0%5D.Name=%D0%BF%D0%BE%D0%B1%D0%B5%D0%B4%D0%B8%D1%82%D0%B5%D0%BB%D1%8C+%D0%BA%D0%BE%D0%BD%D0%BA%D1%83%D1%80%D1%81%D0%B0&Statuses%5B1%5D.Name=%D0%BD%D0%B0+%D0%BD%D0%B5%D0%B7%D0%B0%D0%B2%D0%B8%D1%81%D0%B8%D0%BC%D0%BE%D0%B9+%D1%8D%D0%BA%D1%81%D0%BF%D0%B5%D1%80%D1%82%D0%B8%D0%B7%D0%B5&Statuses%5B2%5D.Name=%D0%BF%D1%80%D0%BE%D0%B5%D0%BA%D1%82+%D0%BD%D0%B5+%D0%BF%D0%BE%D0%BB%D1%83%D1%87%D0%B8%D0%BB+%D0%BF%D0%BE%D0%B4%D0%B4%D0%B5%D1%80%D0%B6%D0%BA%D1%83&RegionId=&AreaCityId=&ContestDirectionTenantId=36568399-3aff-45f4-b418-d5362caf4595&IsNormalTermProjects=true&IsLongTermProjects=true&CompetitionId=&DateFrom=&DateTo=&Statuses%5B0%5D.Selected=false&Statuses%5B1%5D.Selected=false&Statuses%5B2%5D.Selected=false&IsNormalTermProjects=false&IsLongTermProjects=false'

In [126]:
browser = webdriver.Chrome(ChromeDriverManager().install(), ) # options=chrome_options
timeout = 10

In [5]:
def scrape_links():
    table_project = browser.find_element_by_xpath("//div[@class='table table--p-present table--projects']")
    project_links = []
    for i in table_project.find_elements_by_tag_name('a'):
        project_links.append(i.get_attribute('href'))
    return project_links

In [129]:
def scrape_elems(link):
    
    # load new page
    browser.get(link)
    WebDriverWait(browser, timeout).until(
                    lambda browser: browser.find_element(By.XPATH, "//div[@id='winner-aims']"))
    time.sleep(1)
    
    status = browser.find_element_by_xpath("//p[@class='winner-info__status']//span").text
    title = browser.find_element_by_tag_name('h2').text
    
    # сбор информации в content winner
    info = browser.find_element_by_xpath("//ul[@class='winner-info__list winner-info__item']")
    info_elems = info.find_elements_by_xpath("//span[@class='winner-info__list-item-text']")
    
    # если проект получил финансирование, то в контенте есть поле рейтинга, которое не нужно
    if len(info_elems) == 8:
        competition_year = info_elems[0].text
        grant_direction = info_elems[1].text
        app_number = info_elems[2].text
        app_date = info_elems[3].text
        implementation_terms = info_elems[4].text
        organization = info_elems[5].text
        inn = info_elems[6].text
        ogrn = info_elems[7].text
    
    else:
        competition_year = info_elems[0].text
        grant_direction = info_elems[1].text
        app_number = info_elems[3].text
        app_date = info_elems[4].text
        implementation_terms = info_elems[5].text
        organization = info_elems[6].text
        inn = info_elems[7].text
        ogrn = info_elems[8].text
        
    # сбор информации о сумме гранта    
    money = browser.find_element_by_xpath("//ul[@class='circle-bar__info']")
    money_elems = money.find_elements_by_xpath("//span[@class='circle-bar__info-item-number']")
    requested_money = money_elems[0].text
    cofinancing = money_elems[1].text
    total_money = money_elems[2].text
    
    
    # сбор информации с кратким описанием проекта
    summary = browser.find_element_by_xpath("//div[@id='winner-summary']").text.split('\n')[1]
    target = browser.find_element_by_xpath("//div[@id='winner-aims']").find_element_by_tag_name('li').text
    
    return status, title, link, competition_year, grant_direction, app_number, app_date, \
            implementation_terms, organization, inn, ogrn, summary, target, requested_money, cofinancing, total_money

In [130]:
def project_data_to_df(data, out_df):
    cols = ['status', 'title', 'links', 'competition_year', 'grant_direction', 'app_number', 'app_date', \
            'implementation_terms', 'organization', 'inn', 'ogrn', 'summary', 'target', 'requested_money', 'cofinancing', 'total_money']
    if out_df is None:
        out_df = pd.DataFrame(data, columns=cols)
    else:
        out_df = pd.concat([out_df, pd.DataFrame(data, columns=cols)])
    return out_df

In [53]:
proj_data = []
final_df = None

for page in tqdm(range(99, 191)): # 89 -> 191
    start_url = f'https://президентскиегранты.рф/public/application/cards?SearchString=&Statuses%5B0%5D.Name=%D0%BF%D0%BE%D0%B1%D0%B5%D0%B4%D0%B8%D1%82%D0%B5%D0%BB%D1%8C+%D0%BA%D0%BE%D0%BD%D0%BA%D1%83%D1%80%D1%81%D0%B0&Statuses%5B1%5D.Name=%D0%BD%D0%B0+%D0%BD%D0%B5%D0%B7%D0%B0%D0%B2%D0%B8%D1%81%D0%B8%D0%BC%D0%BE%D0%B9+%D1%8D%D0%BA%D1%81%D0%BF%D0%B5%D1%80%D1%82%D0%B8%D0%B7%D0%B5&Statuses%5B2%5D.Name=%D0%BF%D1%80%D0%BE%D0%B5%D0%BA%D1%82+%D0%BD%D0%B5+%D0%BF%D0%BE%D0%BB%D1%83%D1%87%D0%B8%D0%BB+%D0%BF%D0%BE%D0%B4%D0%B4%D0%B5%D1%80%D0%B6%D0%BA%D1%83&RegionId=&AreaCityId=&ContestDirectionTenantId=36568399-3aff-45f4-b418-d5362caf4595&IsNormalTermProjects=true&IsLongTermProjects=true&CompetitionId=&DateFrom=&DateTo=&Statuses%5B0%5D.Selected=false&Statuses%5B1%5D.Selected=false&Statuses%5B2%5D.Selected=false&IsNormalTermProjects=false&IsLongTermProjects=false&page={page}'
    
    browser.get(start_url)
    WebDriverWait(browser, timeout).until(
                        lambda browser: browser.find_element(By.XPATH, "//div[@class='table table--p-present table--projects']"))

    time.sleep(5)
    links = scrape_links()

    for link in tqdm(links):
        data = scrape_elems(link)
        proj_data.append(data)
    
    final_df = project_data_to_df(proj_data, final_df)
    final_df.to_csv('data/scrape-09-09.csv')
    proj_data.clear()

  0%|          | 0/92 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

...

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/13 [00:00<?, ?it/s]

In [59]:
189 * 20 + len(links)

3793

In [60]:
final_df.shape

(3793, 13)

In [64]:
final_df.head()

,status,title,links,competition_year,grant_direction,app_number,app_date,implementation_terms,organization,inn,ogrn,summary,target
0,проект не получил поддержку,Проект по Социо-культурная адаптация детей миг...,https://xn--80afcdbalict6afooklqi5o.xn--p1ai/p...,Второй конкурс 2022,Укрепление межнационального и межрелигиозного ...,22-2-012794,14.03.2022,01.08.2022 - 01.12.2022,ЗАБАЙКАЛЬСКОЕ РЕГИОНАЛЬНОЕ ОТДЕЛЕНИЕ ОБЩЕРОССИ...,7536114997,1107500000772,"Социо-культурная адаптация детей мигрантов, пр...","Создание, внедрение и отработка регламента, пр..."
1,проект не получил поддержку,"Адаптация студентов, прибывающих в Россию из т...",https://xn--80afcdbalict6afooklqi5o.xn--p1ai/p...,Второй конкурс 2022,Укрепление межнационального и межрелигиозного ...,22-2-017763,15.03.2022,01.07.2022 - 31.01.2023,"АВТОНОМНАЯ НЕКОММЕРЧЕСКАЯ ОРГАНИЗАЦИЯ ""ЦЕНТР П...",7321013526,1217300000037,"Цель проекта «Адаптация студентов, прибывающих...",Содействовать формированию благоприятной межку...
2,проект не получил поддержку,По улицам города пройдусь,https://xn--80afcdbalict6afooklqi5o.xn--p1ai/p...,Второй конкурс 2022,Укрепление межнационального и межрелигиозного ...,22-2-007295,15.03.2022,01.07.2022 - 30.11.2022,"ОБЩЕСТВЕННАЯ ОРГАНИЗАЦИЯ ""ОРЕНБУРГСКАЯ РЕГИОНА...",5638032323,1085600000640,"Проект ""По улицам города пройдусь"" - флагман г...",Повысить уровень гражданско-патриотического во...
3,победитель конкурса,"Культурно-этнографическая киноэкспедиция ""Дети...",https://xn--80afcdbalict6afooklqi5o.xn--p1ai/p...,Второй конкурс 2022,Укрепление межнационального и межрелигиозного ...,22-2-010603,15.03.2022,01.07.2022 - 31.05.2023,"АВТОНОМНАЯ НЕКОММЕРЧЕСКАЯ ОРГАНИЗАЦИЯ ""ЦЕНТР С...",9709060278,1207700094964,Актуальная проблема сохранения и развития куль...,"Успеть запечатлеть и зафиксировать, а уже отча..."
4,проект не получил поддержку,Дорога домой,https://xn--80afcdbalict6afooklqi5o.xn--p1ai/p...,Второй конкурс 2022,Укрепление межнационального и межрелигиозного ...,22-2-012647,15.03.2022,01.07.2022 - 31.05.2023,ОБЩЕСТВЕННАЯ ОРГАНИЗАЦИЯ БЕЛОРУССКОЙ МЕСТНОЙ Н...,6154142630,1166100050270,"На базе центра национальных культур""Вместе дру...",Повысить интерес к славянской культуре и народ...


In [87]:
final_df.duplicated().sum()

0

In [67]:
final_df.reset_index(drop=True, inplace=True)

In [70]:
final_df.to_csv('data/scrape-08-09.csv')

In [72]:
pd.read_csv('data/scrape-08-09.csv', index_col=0).shape

(3793, 13)

In [90]:
all_links = []

for page in tqdm(range(1, 191)): # 89 -> 191
    start_url = f'https://президентскиегранты.рф/public/application/cards?SearchString=&Statuses%5B0%5D.Name=%D0%BF%D0%BE%D0%B1%D0%B5%D0%B4%D0%B8%D1%82%D0%B5%D0%BB%D1%8C+%D0%BA%D0%BE%D0%BD%D0%BA%D1%83%D1%80%D1%81%D0%B0&Statuses%5B1%5D.Name=%D0%BD%D0%B0+%D0%BD%D0%B5%D0%B7%D0%B0%D0%B2%D0%B8%D1%81%D0%B8%D0%BC%D0%BE%D0%B9+%D1%8D%D0%BA%D1%81%D0%BF%D0%B5%D1%80%D1%82%D0%B8%D0%B7%D0%B5&Statuses%5B2%5D.Name=%D0%BF%D1%80%D0%BE%D0%B5%D0%BA%D1%82+%D0%BD%D0%B5+%D0%BF%D0%BE%D0%BB%D1%83%D1%87%D0%B8%D0%BB+%D0%BF%D0%BE%D0%B4%D0%B4%D0%B5%D1%80%D0%B6%D0%BA%D1%83&RegionId=&AreaCityId=&ContestDirectionTenantId=36568399-3aff-45f4-b418-d5362caf4595&IsNormalTermProjects=true&IsLongTermProjects=true&CompetitionId=&DateFrom=&DateTo=&Statuses%5B0%5D.Selected=false&Statuses%5B1%5D.Selected=false&Statuses%5B2%5D.Selected=false&IsNormalTermProjects=false&IsLongTermProjects=false&page={page}'
    
    browser.get(start_url)
    WebDriverWait(browser, timeout).until(
                        lambda browser: browser.find_element(By.XPATH, "//div[@class='table table--p-present table--projects']"))

    time.sleep(7)
    links = scrape_links()
    all_links.append(links)

  0%|          | 0/190 [00:00<?, ?it/s]

In [91]:
all_links = [c for i in all_links for c in i]

In [131]:
proj_data = []
final_df = None

save_steps = 0
for link in tqdm(all_links):
    data = scrape_elems(link)
    proj_data.append(data)
    
    save_steps += 1
    if save_steps == 20:
        final_df = project_data_to_df(proj_data, final_df)
        final_df.to_csv('data/scrape-10-09.csv')
        proj_data.clear()
        save_steps = 0
        
final_df = project_data_to_df(proj_data, final_df)
final_df.to_csv('data/scrape-10-09.csv')

  0%|          | 0/3793 [00:00<?, ?it/s]

In [132]:
final_df.duplicated().sum()

0

In [133]:
final_df.reset_index(drop=True, inplace=True)

In [134]:
final_df.to_csv('data/scrape-10-09.csv')

In [135]:
final_df.sample(12)

,status,title,links,competition_year,grant_direction,app_number,app_date,implementation_terms,organization,inn,ogrn,summary,target,requested_money,cofinancing,total_money
3035,проект не получил поддержку,Межнациональный театр для детей «Дом Дружбы»,https://xn--80afcdbalict6afooklqi5o.xn--p1ai/p...,Первый конкурс 2018,Укрепление межнационального и межрелигиозного ...,18-1-005479,19.03.2018,01.07.2018 - 31.08.2019,Волгоградская областная общественная организац...,3444118352,1043400007243,Наш проект «Межнациональный театр для детей «Д...,1. Воспитание детей дошкольного возраста в дух...,"1 389 070,00 ₽","307 444,00 ₽","1 696 514,00 ₽"
736,проект не получил поддержку,Культурный код многонационального региона,https://xn--80afcdbalict6afooklqi5o.xn--p1ai/p...,Первый конкурс 2022,Укрепление межнационального и межрелигиозного ...,22-1-000998,13.10.2021,01.02.2022 - 30.11.2022,САРАТОВСКАЯ РЕГИОНАЛЬНАЯ ОБЩЕСТВЕННАЯ ОРГАНИЗА...,6450049712,1036405000444,"СРОО НКЦ ""Кавказ"" представляет проект ""Культур...","Укрепление дружбы, межнационального и межрелиг...","748 645,00 ₽","460 856,00 ₽","1 209 501,00 ₽"
292,проект не получил поддержку,"Словарь-приложение ""Слова любви на разных язык...",https://xn--80afcdbalict6afooklqi5o.xn--p1ai/p...,Второй конкурс 2022,Укрепление межнационального и межрелигиозного ...,22-2-018887,15.03.2022,01.07.2022 - 30.06.2023,"АВТОНОМНАЯ НЕКОММЕРЧЕСКАЯ ОРГАНИЗАЦИЯ ""ЦЕНТР И...",1657245802,1181690038860,Словарь-приложение «Слова любви на разных язык...,До 1 июля 2023 г. разработать и сделать доступ...,"669 669,00 ₽","212 520,00 ₽","882 189,00 ₽"
1281,проект не получил поддержку,Единый консультационный центр по вопросам прие...,https://xn--80afcdbalict6afooklqi5o.xn--p1ai/p...,Первый конкурс 2021,Укрепление межнационального и межрелигиозного ...,21-1-015608,14.10.2020,01.02.2021 - 31.07.2022,Приморская региональная общественная организац...,2536970082,1112500001547,В 2021-2022 годах к переселению в Приморье гот...,"Оказать 17 видов услуг консультационного, инфо...","2 725 890,00 ₽","650 879,00 ₽","3 376 769,00 ₽"
2588,проект не получил поддержку,II- Всероссийский молодежный форум «Фортанга-2...,https://xn--80afcdbalict6afooklqi5o.xn--p1ai/p...,Второй конкурс 2018,Укрепление межнационального и межрелигиозного ...,18-2-021960,10.09.2018,17.12.2018 - 12.08.2019,Чеченская республиканская организация Общеросс...,2013430029,1052000000469,"II- Всероссийский молодёжный форум ""Фортанга –...",Создание постоянно действующей социально-образ...,"2 998 100,00 ₽","1 636 400,00 ₽","4 634 500,00 ₽"
3105,проект не получил поддержку,"""Союз - Наследники Победы"" ( молодежные образо...",https://xn--80afcdbalict6afooklqi5o.xn--p1ai/p...,Второй конкурс 2017,Укрепление межнационального и межрелигиозного ...,17-2-002044,09.09.2017,01.12.2017 - 10.05.2018,Региональное общественное движение поддержки з...,7726339195,1057700029342,"Развитие сотрудничества в сфере культуры, обра...",Содействие духовно-нравственному воспитанию по...,"7 060 167,00 ₽","22 585 500,07 ₽","29 645 667,07 ₽"
2169,победитель конкурса,Дом семейных традиций,https://xn--80afcdbalict6afooklqi5o.xn--p1ai/p...,Первый конкурс 2019,Укрепление межнационального и межрелигиозного ...,19-1-032709,15.03.2019,01.08.2019 - 30.09.2020,Златоустовская городская женская общественная ...,7404012212,1027400003510,"Проект ""Дом семейных традиций"" решает задачи п...",Цель проекта – создание условий для развития к...,"499 960,00 ₽","796 386,00 ₽","499 960,00 ₽"
279,победитель конкурса,Культурный код многонационального региона,https://xn--80afcdbalict6afooklqi5o.xn--p1ai/p...,Второй конкурс 2022,Укрепление межнационального и межрелигиозного ...,22-2-008950,14.03.2022,01.07.2022 - 30.12.2022,САРАТОВСКАЯ РЕГИОНАЛЬНАЯ ОБЩЕСТВЕННАЯ ОРГАНИЗА...,6450049712,1036405000444,"СРОО НКЦ ""Кавказ"" представляет проект ""Культур...","Укрепление дружбы, межнационального и межрелиг...","499 680,00 ₽","325 410,00 ₽","499 680,00 ₽"
3405,проект не получил поддержку,Сотрудничество во имя единства. Развитие межн